In [ ]:
import pandas as pd

In [ ]:
# getting the raw data:
raw_data = pd.read_csv("./sample_raw_input.csv",
                      nrows=1)
## all rows below got NaN and weird values

raw_data.head()

In [ ]:
# make it long format:
long_data = pd.melt(raw_data, var_name="pathways",value_name="amount")

In [ ]:
long_data

In [ ]:
# now do some RegEx to format for networkx graph:

# format for: nx.from_pandas_edgelist(df,source,target,attr)

# 1. make basic DF with specified col name:
dfInput4nx = pd.DataFrame(columns=["source","target","weight"])


In [ ]:
long_data["pathways"]

In [ ]:
import re

m = re.search("[^(and)]", "L.lysine.fermentation.to.acetate.and.butanoate")

m.group()


In [ ]:
# compiling regex:

# in case of 'to':
p_to_source = re.compile("\S+?(?=\.+to)")
p_to_target = re.compile("(?<=\.to.)\S+")

# target has 'and':
p_and_firstT = re.compile("\S+(?=\.and\.)")
p_and_secondT = re.compile("(?<=\.and\.)\S+")

# sourse: just get the metabolite:
s_no_ion = re.compile("\S+(?=\.\w+ion)")

# replacing biosynthesis only's:
t_no_synth4sub = re.compile("\.\w+synthesis\S*")

# getting rid of roman numerals:
p_no_IV = re.compile("\.[IV]*$")

# getting before & after 'from':
t_from = re.compile("\S+(?=\.\S*from\.\S*)")
s_from = re.compile("(?<=from\.)\S+")

In [ ]:
## testing 
wordT = "acetate.and.butanoate"
re.search(p_and_firstT,wordT)

wordS = "L.lysine.fermentation.to"
re.search(s_no_ion,wordS)

synth = "L.lysine.biosynthesis.III"
re.search(t_no_synth4sub,synth)

re.search(s_no_ion,"acetate.formation")

re.search(t_from,"acetate.formation.from.acetyl.CoA")

In [ ]:
# 1. make basic DF with specified col name:
dfInput4nx = pd.DataFrame(columns=["source","target","weight"])

# 2. RegEx the pathway names to get source and target and fill in weight accordingly

## brute forcing lol:
for rowNum in range(len(long_data)):
    path=long_data["pathways"][rowNum]
    print(rowNum,path)
    weight = long_data["amount"][rowNum]
    
    # if path has roman numerals or period attached:
    if re.search(p_no_IV,path):
        path=re.sub(p_no_IV,"",path)
    
    # if 'to' format:
    if re.search("\.to\.", path):
        source = re.search(p_to_source,path).group()
        # if source has -ion attached:
        if re.search(s_no_ion,source):
            source = re.search(s_no_ion,source).group()
            
        target = re.search(p_to_target,path).group()
        
        # if resulting target more than 1:
        if re.search("and",target):
            #print(target)
            #print(re.search(p_and_firstT,target))
            target1=re.search(p_and_firstT,target).group()
            target2=re.search(p_and_secondT,target).group()
            
            dfInput4nx= dfInput4nx.append({"source":source, "target":target1, "weight":weight},ignore_index=True)
            dfInput4nx= dfInput4nx.append({"source":source, "target":target2, "weight":weight},ignore_index=True)
            
        # if target is just 1:
        else:
            # adding all the info to the df:
            dfInput4nx= dfInput4nx.append({"source":source, "target":target, "weight":weight},ignore_index=True)
            
    # if it has 'synthesis' but no 'to':
    if re.search(t_no_synth4sub,path):
        source = "unknown"+str(rowNum)
        target = re.sub(t_no_synth4sub,"",path)
        dfInput4nx=dfInput4nx.append({"source":source, "target":target, "weight":weight},ignore_index=True)
        
    # if it's 'from' format:
    if re.search("from",path):
        source = re.search(s_from,path).group()
        
        target = re.search(t_from,path).group()
        # if target has -ion attached:
        if re.search(s_no_ion,target):
            target = re.search(s_no_ion,target).group()
            
        dfInput4nx =dfInput4nx.append({"source":source, "target":target, "weight":weight}, ignore_index=True)
        
        print(path,source,target,weight)

    
dfInput4nx    

In [ ]:
import network_analysis as nt

In [ ]:
from imp import reload
reload(nt)

In [ ]:
graph = nt.initialize_graph(dfInput4nx)

In [ ]:
nt.draw_network_graph(graph)

In [ ]:
import matplotlib.pyplot as plt
plt.savefig('blue_MultiDiSpring_sample1.png')

In [ ]:
def draw_network_graph(graph):
    """
    Plots the network graph taken from the input
    """
    import networkx as nx
    import matplotlib.pyplot as plt
    import random
    
    for u,v,d in graph.edges(data=True):
        d['weight'] = random.random()

    edges,weights = zip(*nx.get_edge_attributes(graph,'weight').items())

    
    nx.draw_networkx(graph, 
                     with_labels=True,
                     arrows=True, 
                     arrowsize=20,
                     node_size=900,
                     node_color='skyblue',
                     width=6,
                    pos=nx.spring_layout(graph,k=4),
                    edge_cmap=plt.cm.GnBu,
                    edge_color=weights,
                    edgelist=edges,
                    font_size=14)
    
    #plt.figure(figsize=(60,40))
    ax = plt.gca()
    plt.axis("off")
    ax.margins(0.08)
    
    plt.tight_layout()
    plt.show()


In [ ]:
draw_network_graph(graph)
plt.savefig('blue_MultiDiSpring_sample1.png')

In [ ]:
graph.nodes
graph.edges(data=True)


In [ ]:
graph.is_directed()

In [ ]:
graph.is_multigraph()

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.figs